# 01 Introductionary Experiments
In this codebook, we will perform the basic experiments to motivate the use of disjoint generation.

Disjoint as opposed to single-synthesis generation remains largely unexplored in the literature. Single-synthesis generation seems like the obvious choice for most applications, because having access to all variables may allow the model to better capture the underlying distribution. However, as we shall show, disjoint generation has many benefits such as computational speedup, better generalization, and better privacy. Additionally, distributing the generation process across multiple models can allow for use of specialised models for different variable types.

In [1]:
import time
import pandas as pd

In [2]:
df_train = pd.read_csv('datasets/breast_cancer_train.csv')
df_test = pd.read_csv('datasets/breast_cancer_test.csv')

df_train.head()

,Age,Race,Marital Status,T Stage,N Stage,Sixth Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,45,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,15,Positive,Positive,24,1,80,Alive
1,61,White,Married,T1,N1,IIA,Moderately differentiated,2,Regional,17,Positive,Positive,8,1,48,Alive
2,61,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,28,Positive,Negative,32,5,61,Alive
3,40,White,Married,T3,N2,IIIA,Moderately differentiated,2,Regional,60,Positive,Positive,14,5,53,Dead
4,45,White,Married,T1,N1,IIA,Well differentiated,1,Regional,14,Positive,Positive,13,1,94,Alive


In [5]:
# Establish the baseline with single-synthesis with a bayesian network model (probably the worst case classical ML model in terms of time complexity)
from disjoint_generative_model.utils.generative_model_adapters import generate_synthetic_data

start = time.time()
df_bn = generate_synthetic_data(df_train,'datasynthesizer')
end = time.time()

df_bn.to_csv('datasets/breast_cancer_datasynthesizer.csv', index=False)

print('Time taken for single-synthesis with bayesian network model: ', end-start)
# multiple repeats: 27.05, 25.92, 25.97, 26.30, 24.31

================ Constructing Bayesian Network (BN) ================
Adding ROOT Regional Node Examined
Adding attribute Reginol Node Positive
Adding attribute N Stage
Adding attribute Sixth Stage
Adding attribute T Stage
Adding attribute Tumor Size
Adding attribute Age
Adding attribute Survival Months
Adding attribute Marital Status
Adding attribute Status
Adding attribute differentiate
Adding attribute Grade
Adding attribute Race
Adding attribute Progesterone Status
Adding attribute Estrogen Status
Adding attribute A Stage
========================== BN constructed ==========================
Time taken for single-synthesis with bayesian network model:  25.398772716522217


In [6]:
# Establish the baseline with single-synthesis with a CART model (the fastest classical ML model we encountered)
from disjoint_generative_model.utils.generative_model_adapters import generate_synthetic_data

start = time.time()
df_cart = generate_synthetic_data(df_train,'synthpop')
end = time.time()

df_cart.to_csv('datasets/breast_cancer_synthpop.csv', index=False)

print('Time taken for single-synthesis with CART model: ', end-start)
# multiple repeats: 4.18, 4.25, 4.06, 4.24, 4.24

Find out more at https://www.synthpop.org.uk/



Variable(s): Race, Marital.Status, T.Stage, N.Stage, Sixth.Stage, differentiate, A.Stage, Estrogen.Status, Progesterone.Status, Status have been changed for synthesis from character to factor.
Consider changing them to factors. You can do it using parameter 'minnumlevels'.
Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_synthpop.txt 
Time taken for single-synthesis with CART model:  4.018636226654053


In [ ]:
# Establish the baseline with single-synthesis with a GAN model
from disjoint_generative_model.utils.generative_model_adapters import generate_synthetic_data

start = time.time()
df_gan = generate_synthetic_data(df_train,'ctgan')
end = time.time()

df_gan.to_csv('datasets/breast_cancer_ctgan.csv', index=False)

print('Time taken for single-synthesis with GAN model: ', end-start)
# multiple repeats: 452.69, 919.70, 1206.95

/home/lautrup/sdg_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2024-11-20T15:50:54.623156+0100][743091][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
 57%|█████▋    | 1149/2000 [19:55<14:45,  1.04s/it] 

Time taken for single-synthesis with GAN model:  1206.9541540145874


In [ ]:
# Setup the disjoint generative model with the same dataset
from disjoint_generative_model import DisjointGenerativeModels

start = time.time()     # By default, the joining scheme is concatenation (random)
dgms = DisjointGenerativeModels(df_train,['synthpop','synthpop'])   
df_dgms = dgms.fit_generate()   
end = time.time()

df_dgms.to_csv('datasets/breast_cancer_dgms_random.csv', index=False)

print('Time taken for disjoint generative models: ', end-start)
# multiple repeats: 5.88, 5.72, 5.84

Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): A.Stage, Progesterone.Status, N.Stage, Status numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_synthpop.txt 


Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): Estrogen.Status, Race numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_synthpop.txt 
Time taken for disjoint generative models:  5.8448545932769775


In [8]:
# Setup the evaluation methodology 
from syntheval import SynthEval

metrics = {
    "corr_diff" : {"mixed_corr": True},
    "auroc_diff" : {"model": "rf_cls"},
    "cls_acc"   : {"F1_type": "macro"},
    "eps_risk"  : {},
}

dfs = {
    'datasynthesizer': pd.read_csv('datasets/breast_cancer_datasynthesizer.csv'),
    'synthpop': pd.read_csv('datasets/breast_cancer_synthpop.csv'),
    'ctgan': pd.read_csv('datasets/breast_cancer_ctgan.csv'),
    'dgms_random': pd.read_csv('datasets/breast_cancer_dgms_random.csv')
}

SE = SynthEval(df_train, df_test, verbose=False)
res, rank = SE.benchmark(dfs, analysis_target_var='Status', **metrics, rank_strategy='summation')

res

corr_mat_diff           auroc       cls_F1_diff            \
                        value error     value error       value     error   
dataset                                                                     
datasynthesizer      0.402019   NaN  0.010163   NaN    0.021067   0.00851   
synthpop             0.194283   NaN  0.011891   NaN    0.013569  0.007525   
ctgan                1.111686   NaN  0.056804   NaN    0.109858   0.00763   
dgms_random          1.436618   NaN  0.011438   NaN    0.019345  0.008425   

                cls_F1_diff_hout           eps_identif_risk        \
                           value     error            value error   
dataset                                                             
datasynthesizer         0.013658  0.004802          0.45563   NaN   
synthpop                0.013176  0.004996          0.59657   NaN   
ctgan                   0.115485  0.015627         0.400075   NaN   
dgms_random             0.031602  0.015937         0.342655   NaN   

                priv_loss_eps            rank    u_rank    p_rank  
                        value error                                
dataset                                                            
datasynthesizer       0.16182   NaN  5.334312  3.951762   1.38255  
synthpop             0.269575   NaN    5.0936  3.959745  1.133855  
ctgan                  0.1305   NaN  5.178015  3.708589  1.469426  
dgms_random          0.107755   NaN  5.475234  3.925644   1.54959

## Controlling for randomness
Even with the default concatentaion joining scheme, it seems that we are seeing a huge benefit to privacy, at only a small cost to utility. With this anectotically establised there are many experiments we should do to confirm this. We will start by investigating the variation of quality in the data due to random subdivisions of the columns alongside the random joining.

### Experiment 1: Random Subdivision into equal parts
In this experiment we do repeated generation of the same dataset using the same models, but with different random subdivisions of the columns. We will then compare the quality of the generated data using the same metrics as before.

In [ ]:
from syntheval import SynthEval
from itertools import product
from disjoint_generative_model import DisjointGenerativeModels

NUM_REPEATS = 3

metrics = {
    "corr_diff" : {"mixed_corr": True},
    "auroc_diff" : {"model": "rf_cls"},
    "cls_acc"   : {"F1_type": "macro"},
    "eps_risk"  : {},
}

models = ['synthpop', 'datasynthesizer', 'ctgan']

times = []
dgms_dict = {}
for i, model in list(product(range(NUM_REPEATS), models)):
    start = time.time()
    dgms = DisjointGenerativeModels(df_train,[model, model])   
    df_dgms = dgms.fit_generate()   
    end = time.time()

    times.append(end-start)
    dgms_dict[f'{model}_{i}'] = df_dgms

SE = SynthEval(df_train, df_test, verbose=False)
res, rank = SE.benchmark(dgms_dict, analysis_target_var='Status', **metrics, rank_strategy='summation')

Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): Race, Progesterone.Status, Status numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_0_synthpop.txt 


Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): Estrogen.Status, N.Stage, A.Stage numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_1_synthpop.txt 
================ Constructing Bayesian Network (BN) ================
Adding ROOT T Stage
================ Constructing Bayesian Network (BN) ================
Adding ROOT Marital Status
Adding attribute Tumor Size
Adding attribute Age
Adding attribute Survival Months
Adding attribute Sixth Stage
Adding attribute N Stage
Adding attribute Status
Adding attribute Reginol Node Positive
Adding attribute Grade
Adding attribute Regional Node Examined
Adding attribute differentiate
Adding attribute A Stage
Adding attribute Race
Adding attribute Progesterone Status
========================== BN co

[2024-11-25T12:48:57.720145+0100][1179366][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
[2024-11-25T12:48:57.815345+0100][1179367][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
 37%|███▋      | 749/2000 [09:44<16:15,  1.28it/s]
Find out more at https://www.synthpop.org.uk/


...

[2024-11-25T12:59:12.738857+0100][1184743][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
[2024-11-25T12:59:12.955432+0100][1184741][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
 37%|███▋      | 749/2000 [08:21<13:56,  1.49it/s]
Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): Race, N.Stage, Status, Progesterone.Status numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_0_synthpop.txt 


Find out more at https://www.synthpop.org.uk/


Consider changing them to factors. You can do it using parameter 'minnumlevels'.

Variable(s): Estrogen.Status, A.Stage numeric but with only 3 or fewer distinct values turned into factor(s) for synthesis.

Synthetic data exported as csv file(s).
Information on synthetic data written to
  /home/lautrup/repositories/disjoint-synthetic-data-generation/synthesis_info_synthpop_temp_1_synthpop.txt 
================ Constructing Bayesian Network (BN) ================
Adding ROOT Race
================ Constructing Bayesian Network (BN) ================
Adding ROOT Survival Months
Adding attribute Age
Adding attribute Tumor Size
Adding attribute Regional Node Examined
Adding attribute Marital Status
Adding attribute Sixth Stage
Adding attribute T Stage
Adding attribute N Stage
Adding attribute Reginol Node Positive
Adding attribute Estrogen Status
Adding attribute A Stage
========================== BN constructed ==========================
Adding attribute differentiate
Adding attribute Grade


[2024-11-25T13:08:15.399661+0100][1187317][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
[2024-11-25T13:08:15.425318+0100][1187316][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
 22%|██▏       | 449/2000 [05:40<19:35,  1.32it/s]


In [6]:
print(times)

res

[4.390730619430542, 17.136704921722412, 593.9686894416809, 3.6662650108337402, 17.34406304359436, 511.2317461967468, 4.5235512256622314, 26.698811292648315, 350.2445001602173]


corr_mat_diff           auroc       cls_F1_diff            \
                          value error     value error       value     error   
dataset                                                                       
synthpop_0             2.125247   NaN  0.005504   NaN    0.014278  0.008249   
datasynthesizer_0      1.160832   NaN  0.019208   NaN    0.022066  0.008552   
ctgan_0                2.653008   NaN  0.130907   NaN     0.23393  0.009211   
synthpop_1             2.551341   NaN  0.028732   NaN    0.019998  0.008712   
datasynthesizer_1      2.019936   NaN   0.01716   NaN     0.01682  0.008174   
ctgan_1                2.170008   NaN  0.204638   NaN     0.25388  0.007676   
synthpop_2              1.18446   NaN  0.008992   NaN    0.025971  0.008363   
datasynthesizer_2      2.367455   NaN  0.132487   NaN    0.230625   0.00836   
ctgan_2                2.085125   NaN  0.120284   NaN    0.260141  0.007292   

                  cls_F1_diff_hout           eps_identif_risk        \
                             value     error            value error   
dataset                                                               
synthpop_0                0.011222  0.004756         0.217002   NaN   
datasynthesizer_0         0.019962  0.006443         0.413497   NaN   
ctgan_0                   0.210169  0.037832         0.110365   NaN   
synthpop_1                0.017652   0.01075         0.187919   NaN   
datasynthesizer_1         0.026643  0.011433         0.237509   NaN   
ctgan_1                   0.234362  0.028539          0.15921   NaN   
synthpop_2                0.015773  0.009322         0.391872   NaN   
datasynthesizer_2         0.212417   0.03202         0.186055   NaN   
ctgan_2                   0.234294  0.030191         0.228561   NaN   

                  priv_loss_eps            rank    u_rank    p_rank  
                          value error                                
dataset                                                              
synthpop_0               0.0522   NaN  5.682083  3.951285  1.730798  
datasynthesizer_0      0.134228   NaN  5.381364   3.92909  1.452274  
ctgan_0                0.014169   NaN  5.278352  3.402885  1.875466  
synthpop_1             0.056301   NaN  5.668135  3.912356  1.755779  
datasynthesizer_1      0.054437   NaN  5.630598  3.922544  1.708054  
ctgan_1                0.020507   NaN   5.10932  3.289037  1.820283  
synthpop_2             0.140194   NaN  5.407328  3.939394  1.467934  
datasynthesizer_2       0.04698   NaN  5.171707  3.404742  1.766965  
ctgan_2                0.050708   NaN  5.088636  3.367905  1.720731

In [ ]:
res_datasynthesizer = res[res['dataset'].str.startswith('datasynthesizer')]
res_synthpop = res[res['dataset'].str.startswith('synthpop')]
res_ctgan = res[res['dataset'].str.startswith('ctgan')]

res_datasynthesizer, res_synthpop, res_ctgan

In [7]:
res.std()

corr_mat_diff     value    0.533358
                  error         NaN
auroc             value    0.073372
                  error         NaN
cls_F1_diff       value    0.118868
                  error    0.000558
cls_F1_diff_hout  value    0.108196
                  error    0.012845
eps_identif_risk  value    0.101673
                  error         NaN
priv_loss_eps     value    0.044531
                  error         NaN
rank                       0.237092
u_rank                     0.299699
p_rank                     0.145567
dtype: object

It seems that there is some variation in the quality of data due to differing subdivisions and different models. This points to two important observations:
 - Randoms subdivision and joining can lead to different quality of data.
 - Among these some are actually quite good.

This is a good sign, as it means that we can potentially find a good subdivision and joining that will lead to high quality data. Raising further research questions such as ''how can records be joined such that the quality of the data is maximized?'' and ''does other methods of subdivision than random lead to better quality data?''.

Before moving on to answer any of these questions, we will first investigate the effect of different sized subdivisions on the quality of the data.

### Experiment 2: Random Subdivision into different parts
This experiment continues from the previous one, but instead of subdividing the data into equal sized parts, we will randomly subdivide the data into different fractions. We will then compare the quality of the data generated using the same metrics as before.